# Introduction
This file contains the derivation for the kinematics for the contact controller. To make this controller independent of the drone position in the inertial frame (actually, to make this controller totally independent of the inertial frame), a contact frame $\mathcal{F}_{C}$ is established at the point of contact, with its $xy$-plane tangent to the wall and the $z$-axis pointing into the wall. The robot is then regarded as a serial arm where the drone base $B$ is the theoretical end-effector.
The goal of the resulting inverse kinematics module is to determine joint velocities and body rates of $B$ in $\mathcal{F}_{B}$ such that the virtual joint at the contact point is stabilized to a certain state.

Nomenclature is unchanged w.r.t. the omnidrone_kinematics derivation. In the first iteration, we derive the kinematics as if the virtual joint at the TacTip has three degrees of freedom: Linear movement along $z_{C}$ and rotation about $x_{C}$ and $y_{C}$, to reflect the usage of the simple_cnn for pose estimation.

In [4]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
Zs, alphas, betas, q1, q2, q3 = sp.symbols('z_s alpha_s beta_s q_1 q_2 q_3', real=True)
joint_vector = sp.Matrix([q1, q2, q3])

In [9]:
import numpy as np
import sympy as sp

class DenavitHartenberg:
    def __init__(self, parameters: np.ndarray):
        """
        Initialize the Denavit-Hartenberg parameters for the robot.

        Parameters:
            parameters (np.ndarray): An (N, 4) array of DH parameters, where each row corresponds to
                                     [theta, alpha, d, r] for one joint.
        """
        self.DHparameters = parameters
        self.number_of_joints = self.DHparameters.shape[0]

    def getTransformations(self):
        """
        Compute and stack homogeneous transformation matrices for the given DH parameters.

        Returns:
            np.ndarray: A 3D array where each slice along the third dimension is a 4x4 transformation matrix.
        """
        # Initialize an empty list to store transformation matrices
        self.transformations = []

        for i in range(self.number_of_joints):
            theta, alpha, d, r = self.DHparameters[i]

            # Compute the transformation matrix for the current joint
            transformation = sp.Matrix([
                [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), r * sp.cos(theta)],
                [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), r * sp.sin(theta)],
                [0, sp.sin(alpha), sp.cos(alpha), d],
                [0, 0, 0, 1]
            ])

            # Append the transformation matrix to the list
            self.transformations.append(np.array(transformation))

        # Stack the list of matrices along the third dimension
        return np.stack(self.transformations, axis=2)

    def getEquivalentTransformation(self, base_transformation = sp.eye(4), end_transformation = sp.eye(4)):
        """
        Compute the equivalent transformation matrix symbolically.
        Returns:
            np.ndarray: A 4x4 transformation matrix representing the equivalent transformation.
        """
        # Initialize an identity matrix to store the equivalent transformation
        equivalent_transformation = sp.eye(4)

        # Compute the equivalent transformation by multiplying all transformation matrices
        for transformation in self.transformations:
            equivalent_transformation = equivalent_transformation @ transformation
        
        # Add a base transformation if provided
        equivalent_transformation = base_transformation @ equivalent_transformation

        #Add an end transformation if provided
        equivalent_transformation = equivalent_transformation @ end_transformation

        return equivalent_transformation

# Example usage
if __name__ == "__main__":
    # Define params for the robot
    L1 = L1_length
    L2 = L2_length
    L3 = L3_length

    # Define DH parameters as an array of [theta, alpha, d, r] for each joint
    dh_params = np.array([
        [0.0, sp.pi/2, -Zs, 0.0],
        [sp.pi/2+betas, sp.pi/2, 0.0, 0.0],
        [alphas, sp.pi/2, 0.0, L3],
        [q3, -sp.pi/2, 0.0, L2],
        [q2, sp.pi/2, 0.0, L1],
        [q1, 0.0, 0.0, 0.0]
    ])

    dh = DenavitHartenberg(dh_params)
    transformations = dh.getTransformations()
    base_transform = sp.Matrix([[1,0,0,0],[0,-1,0,0],[0,0,-1,0], [0,0,0,1]])
    end_transform = sp.Matrix([[0,1,0,0],[0,0,-1,0],[-1,0,0,0], [0,0,0,1]])
    equivalent_transformation = dh.getEquivalentTransformation(base_transform, end_transform)

    print("Stacked Transformation Matrices:")
    sp.pprint(equivalent_transformation)


Stacked Transformation Matrices:
⎡-(-sin(βₛ)⋅cos(αₛ)⋅cos(q₃) + sin(q₃)⋅cos(βₛ))⋅sin(q₂) + sin(αₛ)⋅sin(βₛ)⋅cos(q ↪
⎢                                                                              ↪
⎢                  -sin(αₛ)⋅sin(q₂)⋅cos(q₃) + cos(αₛ)⋅cos(q₂)                  ↪
⎢                                                                              ↪
⎢ (sin(βₛ)⋅sin(q₃) + cos(αₛ)⋅cos(βₛ)⋅cos(q₃))⋅sin(q₂) + sin(αₛ)⋅cos(βₛ)⋅cos(q₂ ↪
⎢                                                                              ↪
⎣                                       0                                      ↪

↪ ₂)  ((-sin(βₛ)⋅cos(αₛ)⋅cos(q₃) + sin(q₃)⋅cos(βₛ))⋅cos(q₂) + sin(αₛ)⋅sin(βₛ)⋅ ↪
↪                                                                              ↪
↪                                    -(-sin(αₛ)⋅cos(q₂)⋅cos(q₃) - sin(q₂)⋅cos( ↪
↪                                                                              ↪
↪ )   -((sin(βₛ)⋅sin(q₃) + cos(αₛ)⋅cos(βₛ)⋅cos(q₃))⋅cos(q₂) - sin(αₛ)⋅sin(q

In [10]:
# Check for nominal configuration
sp.pprint(equivalent_transformation.subs({q1: 0, q2: 0, q3: 0, alphas: 0, betas: 0, Zs: 0}).evalf())


⎡ 0    0    -1.0    0   ⎤
⎢                       ⎥
⎢1.0   0     0      0   ⎥
⎢                       ⎥
⎢ 0   -1.0   0    -0.694⎥
⎢                       ⎥
⎣ 0    0     0     1.0  ⎦


In [11]:
# Extract the FK function


⎡-(-sin(βₛ)⋅cos(αₛ)⋅cos(q₃) + sin(q₃)⋅cos(βₛ))⋅sin(q₂) + sin(αₛ)⋅sin(βₛ)⋅cos(q ↪
⎢                                                                              ↪
⎢                  -sin(αₛ)⋅sin(q₂)⋅cos(q₃) + cos(αₛ)⋅cos(q₂)                  ↪
⎢                                                                              ↪
⎢ (sin(βₛ)⋅sin(q₃) + cos(αₛ)⋅cos(βₛ)⋅cos(q₃))⋅sin(q₂) + sin(αₛ)⋅cos(βₛ)⋅cos(q₂ ↪
⎢                                                                              ↪
⎣                                       0                                      ↪

↪ ₂)  ((-sin(βₛ)⋅cos(αₛ)⋅cos(q₃) + sin(q₃)⋅cos(βₛ))⋅cos(q₂) + sin(αₛ)⋅sin(βₛ)⋅ ↪
↪                                                                              ↪
↪                                    -(-sin(αₛ)⋅cos(q₂)⋅cos(q₃) - sin(q₂)⋅cos( ↪
↪                                                                              ↪
↪ )   -((sin(βₛ)⋅sin(q₃) + cos(αₛ)⋅cos(βₛ)⋅cos(q₃))⋅cos(q₂) - sin(αₛ)⋅sin(q₂)⋅ ↪
↪                          